A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é um proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados cada um em um ranking, da melhor (rank 1) para a pior empresa. Depois somamos os números das duas colunas, quanto menor a soma, melhor.

Para os princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e  seguradoras devem ser excluídas da seleção. 

In [16]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [17]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [18]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 1000000.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [19]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [20]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [21]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.43,85.55,6.165940e+07,1.0,1.0,2.0
2,WIZC3,1.38,29.67,2.619570e+06,2.0,9.0,11.0
3,BOBR4,3.87,37.91,1.098800e+06,13.0,3.0,16.0
4,PLPL3,4.24,43.68,8.609080e+06,15.0,2.0,17.0
5,CMIN3,3.95,36.91,8.233830e+07,18.0,4.0,22.0
6,PETR4,2.49,23.37,1.265610e+09,6.0,17.0,23.0
7,PETR3,2.65,23.37,3.975480e+08,7.0,17.0,24.0
8,KEPL3,4.43,31.43,9.650670e+06,20.0,8.0,28.0
9,VALE3,3.28,19.72,1.430070e+09,9.0,22.0,31.0
10,VLID3,3.51,20.98,1.020380e+07,17.0,20.0,37.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [22]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


RPAD3 RPAD6 RPAD5 BGIP4 BAZA3 BGIP3 BBAS3 PINE3 PINE4 BNBR3 BRSR6 ABCB4 BRSR3 BMEB3 BMGB4 BMEB4 ITSA4 ITSA3 BEES3 BEES4 BRSR5 ITUB3 BRBI11 BPAC5 BBDC3 SANB3 ITUB4 SANB11 SANB4 BBDC4 BMIN4 BPAC11 BMIN3 MERC4 BPAN4 BPAC3 BSLI3 BSLI4
APER3 WIZC3 BBSE3 PSSA3 CXSE3 IRBR3
AMBP3 SAPR3 SAPR11 SAPR4 CSMG3 SBSP3 ORVR3 GPAR3 AESB3 AURE3 CBEE3 LIGT3 CEED4 CEED3 CSRN3 CLSC3 CSRN5 CSRN6 CLSC4 NEOE3 CMIG4 EQMA3B COCE5 CEEB5 TRPL4 EQPA3 CEEB3 COCE3 CMIG3 CPFE3 CEBR5 ENGI4 CEBR6 TRPL3 EGIE3 CEBR3 EQPA5 ENGI11 REDE3 EKTR4 TAEE3 TAEE4 TAEE11 EMAE4 RNEW3 RNEW11 RNEW4 CPLE3 ENMT4 ENGI3 ALUP4 ALUP11 CPLE6 ALUP3 ENMT3 CPLE5 AFLT3 GEPA3 GEPA4 EQTL3 SRNA3 ELET3 ELET6 LIPR3 ENEV3 ELET5


#### sem bancos

In [23]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,36.91,9.61,1.85,0.691,4.48,0.534,9.77,0.84,-4.33,0.44,0.43,82.66,7.30,1.10,85.55,19.25,61659400.0,1.289760e+10,0.00,19.02,1.0,1.0,2.0,PSSA
WIZC3,5.84,5.37,1.70,0.773,4.22,0.369,-80.70,1.53,-1.24,1.66,1.38,50.58,24.39,0.98,29.67,31.70,2619570.0,5.490420e+08,0.65,12.98,2.0,9.0,11.0,WIZC
BOBR4,2.84,7.67,-35.80,0.491,0.00,0.838,-19.43,3.25,-1.71,4.33,3.87,15.13,6.41,0.93,37.91,-467.04,1098800.0,-2.066400e+07,-15.23,10.32,13.0,3.0,16.0,BOBR
PLPL3,13.84,9.55,3.92,1.179,3.65,1.367,2.38,4.37,6.59,4.37,4.24,26.95,13.24,3.15,43.68,41.10,8609080.0,7.192830e+08,0.53,28.26,15.0,2.0,17.0,PLPL
CMIN3,5.93,7.03,2.77,1.802,17.99,1.054,3.78,5.10,-6.10,4.65,3.95,35.33,25.62,2.67,36.91,39.37,82338300.0,1.174650e+10,0.76,-4.26,18.0,4.0,22.0,CMIN
PETR4,36.83,6.10,1.29,0.963,17.21,0.454,-25.74,2.13,-0.92,3.30,2.49,45.15,15.87,0.90,23.37,21.09,1265610000.0,3.734790e+11,0.89,18.69,6.0,17.0,23.0,PETR
KEPL3,9.10,6.32,2.22,1.012,8.73,1.077,2.93,5.72,6.19,5.02,4.43,17.69,16.01,2.15,31.43,35.05,9650670.0,7.382410e+08,0.42,31.15,20.0,8.0,28.0,KEPL
VALE3,60.55,5.64,1.34,1.308,11.83,0.571,45.44,3.33,-1.48,3.91,3.28,39.30,23.32,1.08,19.72,23.72,1430070000.0,2.054510e+11,0.41,2.80,9.0,22.0,31.0,VALE
VLID3,22.68,5.73,1.12,0.846,7.34,0.679,1.92,4.29,5.07,4.44,3.51,19.71,14.68,3.12,20.98,19.58,10203800.0,1.653420e+09,0.36,0.93,17.0,20.0,37.0,VLID
SYNE3,8.50,2.74,0.65,0.917,33.91,0.286,0.85,2.41,5.36,1.72,1.48,38.02,42.31,5.74,17.14,23.57,6825860.0,2.005940e+09,0.50,22.91,3.0,34.0,37.0,SYNE


#### sem seguradora

In [24]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
BOBR4,2.84,7.67,-35.80,0.491,0.00,0.838,-19.43,3.25,-1.71,4.33,3.87,15.13,6.41,0.93,37.91,-467.04,1098800.0,-2.066400e+07,-15.23,10.32,13.0,3.0,16.0,BOBR
PLPL3,13.84,9.55,3.92,1.179,3.65,1.367,2.38,4.37,6.59,4.37,4.24,26.95,13.24,3.15,43.68,41.10,8609080.0,7.192830e+08,0.53,28.26,15.0,2.0,17.0,PLPL
CMIN3,5.93,7.03,2.77,1.802,17.99,1.054,3.78,5.10,-6.10,4.65,3.95,35.33,25.62,2.67,36.91,39.37,82338300.0,1.174650e+10,0.76,-4.26,18.0,4.0,22.0,CMIN
PETR4,36.83,6.10,1.29,0.963,17.21,0.454,-25.74,2.13,-0.92,3.30,2.49,45.15,15.87,0.90,23.37,21.09,1265610000.0,3.734790e+11,0.89,18.69,6.0,17.0,23.0,PETR
KEPL3,9.10,6.32,2.22,1.012,8.73,1.077,2.93,5.72,6.19,5.02,4.43,17.69,16.01,2.15,31.43,35.05,9650670.0,7.382410e+08,0.42,31.15,20.0,8.0,28.0,KEPL
VALE3,60.55,5.64,1.34,1.308,11.83,0.571,45.44,3.33,-1.48,3.91,3.28,39.30,23.32,1.08,19.72,23.72,1430070000.0,2.054510e+11,0.41,2.80,9.0,22.0,31.0,VALE
VLID3,22.68,5.73,1.12,0.846,7.34,0.679,1.92,4.29,5.07,4.44,3.51,19.71,14.68,3.12,20.98,19.58,10203800.0,1.653420e+09,0.36,0.93,17.0,20.0,37.0,VLID
SYNE3,8.50,2.74,0.65,0.917,33.91,0.286,0.85,2.41,5.36,1.72,1.48,38.02,42.31,5.74,17.14,23.57,6825860.0,2.005940e+09,0.50,22.91,3.0,34.0,37.0,SYNE
PRIO3,42.18,6.67,2.12,2.582,0.17,1.009,7.01,4.62,-4.19,5.46,4.17,55.89,38.72,2.04,27.40,31.76,438079000.0,1.776680e+10,0.75,73.92,29.0,11.0,40.0,PRIO
LEVE3,30.28,6.24,4.13,0.958,25.42,1.287,4.05,5.13,-23.58,5.91,5.24,18.65,15.35,2.01,34.32,66.25,12102200.0,9.935900e+08,1.04,18.41,36.0,6.0,42.0,LEVE


#### sem utilidade publica

In [25]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,36.91,9.61,1.85,0.691,4.48,0.534,9.77,0.84,-4.33,0.44,0.43,82.66,7.30,1.10,85.55,19.25,61659400.0,1.289760e+10,0.00,19.02,1.0,1.0,2.0,PSSA
WIZC3,5.84,5.37,1.70,0.773,4.22,0.369,-80.70,1.53,-1.24,1.66,1.38,50.58,24.39,0.98,29.67,31.70,2619570.0,5.490420e+08,0.65,12.98,2.0,9.0,11.0,WIZC
BOBR4,2.84,7.67,-35.80,0.491,0.00,0.838,-19.43,3.25,-1.71,4.33,3.87,15.13,6.41,0.93,37.91,-467.04,1098800.0,-2.066400e+07,-15.23,10.32,13.0,3.0,16.0,BOBR
PLPL3,13.84,9.55,3.92,1.179,3.65,1.367,2.38,4.37,6.59,4.37,4.24,26.95,13.24,3.15,43.68,41.10,8609080.0,7.192830e+08,0.53,28.26,15.0,2.0,17.0,PLPL
CMIN3,5.93,7.03,2.77,1.802,17.99,1.054,3.78,5.10,-6.10,4.65,3.95,35.33,25.62,2.67,36.91,39.37,82338300.0,1.174650e+10,0.76,-4.26,18.0,4.0,22.0,CMIN
PETR4,36.83,6.10,1.29,0.963,17.21,0.454,-25.74,2.13,-0.92,3.30,2.49,45.15,15.87,0.90,23.37,21.09,1265610000.0,3.734790e+11,0.89,18.69,6.0,17.0,23.0,PETR
KEPL3,9.10,6.32,2.22,1.012,8.73,1.077,2.93,5.72,6.19,5.02,4.43,17.69,16.01,2.15,31.43,35.05,9650670.0,7.382410e+08,0.42,31.15,20.0,8.0,28.0,KEPL
VALE3,60.55,5.64,1.34,1.308,11.83,0.571,45.44,3.33,-1.48,3.91,3.28,39.30,23.32,1.08,19.72,23.72,1430070000.0,2.054510e+11,0.41,2.80,9.0,22.0,31.0,VALE
VLID3,22.68,5.73,1.12,0.846,7.34,0.679,1.92,4.29,5.07,4.44,3.51,19.71,14.68,3.12,20.98,19.58,10203800.0,1.653420e+09,0.36,0.93,17.0,20.0,37.0,VLID
SYNE3,8.50,2.74,0.65,0.917,33.91,0.286,0.85,2.41,5.36,1.72,1.48,38.02,42.31,5.74,17.14,23.57,6825860.0,2.005940e+09,0.50,22.91,3.0,34.0,37.0,SYNE


#### sem as 3 classes

In [26]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
BOBR4,2.84,7.67,-35.80,0.491,0.00,0.838,-19.43,3.25,-1.71,4.33,3.87,15.13,6.41,0.93,37.91,-467.04,1098800.0,-2.066400e+07,-15.23,10.32,13.0,3.0,16.0,BOBR
PLPL3,13.84,9.55,3.92,1.179,3.65,1.367,2.38,4.37,6.59,4.37,4.24,26.95,13.24,3.15,43.68,41.10,8609080.0,7.192830e+08,0.53,28.26,15.0,2.0,17.0,PLPL
CMIN3,5.93,7.03,2.77,1.802,17.99,1.054,3.78,5.10,-6.10,4.65,3.95,35.33,25.62,2.67,36.91,39.37,82338300.0,1.174650e+10,0.76,-4.26,18.0,4.0,22.0,CMIN
PETR4,36.83,6.10,1.29,0.963,17.21,0.454,-25.74,2.13,-0.92,3.30,2.49,45.15,15.87,0.90,23.37,21.09,1265610000.0,3.734790e+11,0.89,18.69,6.0,17.0,23.0,PETR
KEPL3,9.10,6.32,2.22,1.012,8.73,1.077,2.93,5.72,6.19,5.02,4.43,17.69,16.01,2.15,31.43,35.05,9650670.0,7.382410e+08,0.42,31.15,20.0,8.0,28.0,KEPL
VALE3,60.55,5.64,1.34,1.308,11.83,0.571,45.44,3.33,-1.48,3.91,3.28,39.30,23.32,1.08,19.72,23.72,1430070000.0,2.054510e+11,0.41,2.80,9.0,22.0,31.0,VALE
VLID3,22.68,5.73,1.12,0.846,7.34,0.679,1.92,4.29,5.07,4.44,3.51,19.71,14.68,3.12,20.98,19.58,10203800.0,1.653420e+09,0.36,0.93,17.0,20.0,37.0,VLID
SYNE3,8.50,2.74,0.65,0.917,33.91,0.286,0.85,2.41,5.36,1.72,1.48,38.02,42.31,5.74,17.14,23.57,6825860.0,2.005940e+09,0.50,22.91,3.0,34.0,37.0,SYNE
PRIO3,42.18,6.67,2.12,2.582,0.17,1.009,7.01,4.62,-4.19,5.46,4.17,55.89,38.72,2.04,27.40,31.76,438079000.0,1.776680e+10,0.75,73.92,29.0,11.0,40.0,PRIO
LEVE3,30.28,6.24,4.13,0.958,25.42,1.287,4.05,5.13,-23.58,5.91,5.24,18.65,15.35,2.01,34.32,66.25,12102200.0,9.935900e+08,1.04,18.41,36.0,6.0,42.0,LEVE


outros filtros

In [27]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [28]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
BOBR4,2.84,7.67,-35.80,0.491,0.00,0.838,-19.43,3.25,-1.71,4.33,3.87,15.13,6.41,0.93,37.91,-467.04,1098800.0,-2.066400e+07,-15.23,10.32,13.0,3.0,16.0,BOBR
PLPL3,13.84,9.55,3.92,1.179,3.65,1.367,2.38,4.37,6.59,4.37,4.24,26.95,13.24,3.15,43.68,41.10,8609080.0,7.192830e+08,0.53,28.26,15.0,2.0,17.0,PLPL
PETR4,36.83,6.10,1.29,0.963,17.21,0.454,-25.74,2.13,-0.92,3.30,2.49,45.15,15.87,0.90,23.37,21.09,1265610000.0,3.734790e+11,0.89,18.69,6.0,17.0,23.0,PETR
KEPL3,9.10,6.32,2.22,1.012,8.73,1.077,2.93,5.72,6.19,5.02,4.43,17.69,16.01,2.15,31.43,35.05,9650670.0,7.382410e+08,0.42,31.15,20.0,8.0,28.0,KEPL
VALE3,60.55,5.64,1.34,1.308,11.83,0.571,45.44,3.33,-1.48,3.91,3.28,39.30,23.32,1.08,19.72,23.72,1430070000.0,2.054510e+11,0.41,2.80,9.0,22.0,31.0,VALE
VLID3,22.68,5.73,1.12,0.846,7.34,0.679,1.92,4.29,5.07,4.44,3.51,19.71,14.68,3.12,20.98,19.58,10203800.0,1.653420e+09,0.36,0.93,17.0,20.0,37.0,VLID
SYNE3,8.50,2.74,0.65,0.917,33.91,0.286,0.85,2.41,5.36,1.72,1.48,38.02,42.31,5.74,17.14,23.57,6825860.0,2.005940e+09,0.50,22.91,3.0,34.0,37.0,SYNE
PRIO3,42.18,6.67,2.12,2.582,0.17,1.009,7.01,4.62,-4.19,5.46,4.17,55.89,38.72,2.04,27.40,31.76,438079000.0,1.776680e+10,0.75,73.92,29.0,11.0,40.0,PRIO
LEVE3,30.28,6.24,4.13,0.958,25.42,1.287,4.05,5.13,-23.58,5.91,5.24,18.65,15.35,2.01,34.32,66.25,12102200.0,9.935900e+08,1.04,18.41,36.0,6.0,42.0,LEVE
LREN3,18.40,15.36,1.72,1.285,3.51,0.921,4.00,6.28,7.94,5.87,4.13,20.47,8.36,1.66,18.26,11.19,371927000.0,1.031710e+10,0.12,15.83,35.0,27.0,62.0,LREN


retirar BDR

In [29]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
40,AURA33,22.9,-29.02,3.6,1.981,4.45,1.014,7.96,7.16,-2.81,...,1.55,20.47,-12.42,5365430.0,1.379760e+09,1.35,8.87,79.0,21.0,100.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [30]:
data_funds.loc['INEP4']

Cotação              1.730000e+00
P/L                  1.600000e-01
P/VP                -6.000000e-02
PSR                  1.172500e+01
Div.Yield            0.000000e+00
P/Ativo              7.000000e-02
P/Cap.Giro          -7.000000e-02
P/EBIT              -1.110000e+00
P/Ativ Circ.Liq     -3.000000e-02
EV/EBIT             -1.416000e+01
EV/EBITDA           -1.591000e+01
Mrg Ebit            -1.052730e+03
Mrg. Líq.            7.127190e+03
Liq. Corr.           9.000000e-02
ROIC                -6.680000e+00
ROE                 -3.656000e+01
Liq.2meses           9.726360e+04
Patrim. Líq         -1.280710e+09
Dív.Brut/ Patrim.   -7.000000e-01
Cresc. Rec.5a        2.860000e+00
Name: INEP4, dtype: float64